<a href="https://colab.research.google.com/github/AlexNoelHdz/convex_optimization/blob/main/Symbolic_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generar nuevas características no lineales usando SymbolicTransformer


Instalar gplearn o librerías no presentes en el servidor. 

In [ ]:
pip install gplearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gplearn.genetic import SymbolicTransformer
from sklearn.utils import check_random_state
from sklearn.datasets import load_diabetes
import numpy as np

Cargar el dataset de Diabetes y definir una semilla para que los datos aleatorios se carguen igual en cada ejecución


In [ ]:
rng = check_random_state(0)
diabetes = load_diabetes()
perm = rng.permutation(diabetes.target.size)
diabetes.data = diabetes.data[perm]
diabetes.target = diabetes.target[perm]

Estás son las features o características del objeto de estudio: 

In [ ]:
diabetes.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

Se entrenará el regresor con las primeras 300 muestras, usando [RIDGE](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) Regression de la librería sklearn. 
Vamos a revisar el comportamiento del modelo con las 300 muestras finales. 
Score compara los datos reales (target data) con los obtenidos con el modelo entrenado y nos da un numero que determina que tan exacta es la predicción.

In [ ]:
from sklearn.linear_model import Ridge
est = Ridge()
# 300 sample data / target data
sample_data = diabetes.data[:300, :]
sample_target_data = diabetes.target[:300]
est.fit(sample_data, sample_target_data)
# score(X: Training Data, y: Target Values) 
# Return the coefficient of determination of the prediction.
print(est.score(diabetes.data[300:, :], diabetes.target[300:]))

0.4340571824299352


El proximo paso es entrenar el transformador [SymbolicTransformer](https://gplearn.readthedocs.io/en/stable/reference.html#symbolic-transformer).

Se utilizarán los 300 datos de muestra con que se entrenó el regresor. 
El objetivo es generar nuevos features. 

En este transformador se usarán: 
  - Población de 2000 individuos a lo largo de 20 generaciones.
  - Seleccionar las mejores 100 muestras (Parámetro: hall_of_fame)
  - Usar las 10 menos correlacionadas como nueva feature (Parámetro: n_components)


In [ ]:
function_set = ['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                'abs', 'neg', 'inv', 'max', 'min']
gp = SymbolicTransformer(generations=20, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0)
gp.fit(sample_data, sample_target_data)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.37         0.126617        5         0.612825         0.680003     40.51s
   1     6.63         0.344374        3         0.659085         0.451792     39.24s
   2     5.36          0.47373        3         0.669018         0.321485     35.44s
   3     4.74         0.587613        3         0.673353         0.312487     32.84s
   4     4.39         0.597168       13         0.675281         0.494818     31.01s
   5     4.31         0.611994       15         0.686134         0.148192     37.39s
   6     4.89         0.611891        9         0.685649         0.199004     29.39s
   7     6.49         0.617031        9         0.688308         0.287285     25.63s
   8     9.19         0.628526       17         0.720605         0.263613  

SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    max_samples=0.9, parsimony_coefficient=0.0005,
                    population_size=2000, random_state=0, verbose=1)

Aplicar la transformación a todo el set de datos con el gp entrenado. 

In [ ]:
gp_features = gp.transform(diabetes.data)
gp_features

array([[ 0.24236152, -0.24866552, -0.38250661, ...,  0.38751358,
        -0.4873153 , -0.39010584],
       [ 0.20064364, -0.23654608, -0.30636002, ...,  0.26846855,
        -0.36697841, -0.23415969],
       [ 0.04846929, -0.02405987, -0.03664407, ...,  0.00391138,
        -0.07973569, -0.01482502],
       ...,
       [ 0.22205692, -0.25126335, -0.49571211, ...,  0.27988268,
        -0.47441679, -0.23813384],
       [-0.09114857,  0.09114857,  0.09114857, ..., -0.09114857,
         0.09114857,  0.09114857],
       [ 0.19971279, -0.23881401, -0.29372021, ...,  0.27921018,
        -0.36511673, -0.24490132]])

Concatenar los nuevos features a la base de datos original.

In [ ]:
new_diabetes = np.hstack((diabetes.data, gp_features))

Entrenar de nuevo Ridge con los datos que ya tienen los nuevos features. Y revisar con los últimos 300 datos el score. 

Observa que incrementa al incrementar la cantidad de features. 

In [ ]:
est = Ridge()
# 300 sample data / target data
sample_data = new_diabetes[:300, :]
sample_target_data = diabetes.target[:300]
est.fit(sample_data, sample_target_data)
print(est.score(new_diabetes[300:, :], diabetes.target[300:]))

0.5336811804650683


Como es posible ver, se ha mejorado significatvamente el score. El modelo lineal due capaz de tomar ventaja de features no lineales para predecir los datos mejor. 

# ¿Por qué symbolic transformer ayuda a mejorar el desempeño de la regresión?

Symbolic transformer implementa un algorítmo genético recursivo que comienza aplicando formulas aleatorias que representan una relación. 
En cada iteración (o generación dado que hablamos de programación genética) se registra una evolución al ajustar el algorítmo con la selección de los individuos más aptos de la población evaluada en la generación anterior, en este punto se crean nuevas features o características. La población final se busca para los individuos más aptos con la menor correlación entre sí.

Es aquí donde Symbolic Transformer ayuda a mejorar el desempeño de la regresión, ya que al reevaluarse con nuevas features que resultan de una transformación de los datos originales tomando los casos más exitosos y disminuyendo la correlación, se logra aumentar el score al entrenar y evaluar de nuevo el modelo. 